In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

In [2]:
def url_request(url):
    driver = webdriver.Chrome()
    driver.get(url)
   
    return driver

In [3]:
driver= url_request('https://www.bbc.com/news')

In [4]:
def keyword(word):
    
    cookies_accept = driver.find_element(By.CLASS_NAME, 'fc-button').click()
    cookies_2= driver.find_element(By.CLASS_NAME, 'banner-button').click()
    
    
    time.sleep(3)
    search_bar  = driver.find_element(By.LINK_TEXT, 'Search BBC').click()
    search_bar  = driver.find_element(By.ID, 'search-input')
    search_bar.send_keys(word)
    search_bar.send_keys(Keys.RETURN)
    
    link_lst=[]
    try:

        while len(link_lst)<10:
            lnks           = driver.find_elements(By.TAG_NAME,"a")
            lst_bbc        = [lnk.get_attribute('href') for lnk in lnks]
            lst_bbc_       = [url for url in lst_bbc if '/news/' in url and '/help' not in url and '/live/' not in url]
            search_bar = driver.find_element(By.LINK_TEXT, 'next page').click()
            for i in lst_bbc_:
                link_lst.append(i)

            link_lst= list(set(link_lst))

            list_soup=[]
            for url in link_lst:
                html = requests.get(url)
                soup = BeautifulSoup(html.content, "html.parser")
                article=soup.getText().replace('\'', "´").strip()

                article = article[:article.find('More on this story')]
                list_soup.append(article)

            day_list=[]
            month_list=[]
            year_list=[]
            for url in link_lst:
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                time_tag = soup.find('time')
                timestamp = time_tag['datetime']
                dt = datetime.fromisoformat(timestamp[:-1])
                date_str = dt.strftime("%d-%m-%Y")


                day, month, year = date_str.split('-')

                day_list.append(day)
                month_list.append(month)
                year_list.append(year)
               



            dict_bbc={'article': list_soup,
                     'link':link_lst,
                     'day':day_list,
                     'month': month_list,
                     'year': year_list}
            df=pd.DataFrame(dict_bbc)
        return df
    except:
        return df


In [5]:
df_bbc=keyword('Russia')

In [6]:
df_bbc

,article,link,day,month,year
0,Russia country profile - BBC NewsBBC HomepageS...,https://www.bbc.co.uk/news/world-europe-17839672,06,03,2023
1,Bakhmut: Fighting in the street but Russia not...,https://www.bbc.co.uk/news/world-europe-64846666,04,03,2023
2,Emmanuel Macron´s mission to counter Russia in...,https://www.bbc.co.uk/news/world-africa-64822485,04,03,2023
3,Russia media guide - BBC NewsBBC HomepageSkip ...,https://www.bbc.co.uk/news/world-europe-17840134,06,03,2023
4,Ukraine war: Russia´s Wagner boss suggests ´be...,https://www.bbc.co.uk/news/world-europe-64859780,06,03,2023
5,Ukraine war: Why Bakhmut matters for Russia an...,https://www.bbc.co.uk/news/world-europe-64877991,09,03,2023
6,Finland starts construction of Russia border f...,https://www.bbc.co.uk/news/world-europe-64802457,28,02,2023
7,US arrests two for illegally exporting aviatio...,https://www.bbc.co.uk/news/world-us-canada-648...,02,03,2023


In [7]:
df_bbc.groupby(['year']).count()

,article,link,day,month
year,,,,
2023,8,8,8,8
